In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import json
from collections import Counter
import pyarrow

warnings.filterwarnings('ignore')

In [3]:
import os
import pandas as pd
import json
import win32com.client

def resolve_shortcut(path):
    """Resuelve un acceso directo de Windows y devuelve la ruta real."""
    shell = win32com.client.Dispatch("WScript.Shell")
    shortcut = shell.CreateShortCut(path)
    return shortcut.Targetpath

def process_directory(directory_path):
    files = os.listdir(directory_path)
    dataframes = []
    
    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(directory_path, file)
            try:
                df = pd.read_json(file_path, lines=True)
                dataframes.append(df)
            except Exception as e:
                print(f"Error al procesar {file_path}: {e}")
    
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        return combined_df
    else:
        return None

created_dataframes = []

for root, dirs, files in os.walk(os.getcwd()):
    for file in files:
        if file.endswith('.lnk'):
            resolved_path = resolve_shortcut(os.path.join(root, file))
            if os.path.isdir(resolved_path):
                dirs.append(resolved_path)  # Agregar la ruta resuelta a la lista de directorios para ser procesada

    for directory in dirs:
        full_directory_path = os.path.join(root, directory)
        df = process_directory(full_directory_path)
        
        if df is not None:
            #df.dropna(inplace=True)
            df_name = directory.replace("-", "_")
            globals()[df_name] = df
            created_dataframes.append(df_name)

print("DataFrames creados:")
for name in created_dataframes:
    print(name)


DataFrames creados:
metadata_sitios
review_Wyoming
review_Wisconsin
review_West_Virginia
review_Washington
review_Virginia
review_Vermont
review_Utah
review_Texas
review_Tennessee
review_South_Dakota
review_South_Carolina
review_Rhode_Island
review_Pennsylvania
review_Oregon
review_Oklahoma
review_Ohio
review_North_Dakota
review_North_Carolina
review_New_York
review_New_Mexico
review_New_Jersey
review_New_Hampshire
review_Nevada
review_Nebraska
review_Montana
review_Missouri
review_Mississippi
review_Minnesota
review_Michigan
review_Massachusetts
review_Maryland
review_Maine
review_Louisiana
review_Kentucky
review_Kansas
review_Iowa
review_Indiana
review_Illinois
review_Idaho
review_Hawaii
review_Georgia
review_Florida
review_District_of_Columbia
review_Delaware
review_Connecticut
review_Colorado
review_California
review_Arkansas
review_Arizona
review_Alaska
review_Alabama


In [4]:
metadata_sitios

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,MemorialCare Breast Center - Irvine Sand Canyon,MemorialCare Breast Center - Irvine Sand Canyo...,0x80dcdd3b95c7284b:0x23d9b4440c982d37,None,33.668389,-117.765649,[Mammography service],4.7,33,None,"[[Tuesday, 8:30AM–5PM], [Wednesday, 8:30AM–5PM...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 8:30AM,"[0x80dcdc4aad8cb91f:0x62bdeb79a44d7633, 0x80dc...",https://www.google.com/maps/place//data=!4m2!3...
2,U-Haul Neighborhood Dealer,"U-Haul Neighborhood Dealer, 1690 E 112th Ave, ...",0x876c769e600eda1d:0xdf1fffc2e00e1fe1,None,39.899189,-104.966765,"[Truck rental agency, Trailer rental service, ...",3.9,18,$,None,None,None,"[0x876c75d05d0e1bf3:0x1d2d9897d0dec9cb, 0x876c...",https://www.google.com/maps/place//data=!4m2!3...
3,Nick's Auto Service,"Nick's Auto Service, 16291 Construction Cir E ...",0x80dcdc81a6d04bd3:0x72d18ce084849c10,None,33.694544,-117.821692,"[Auto repair shop, Brake shop, Car inspection ...",4.3,78,None,"[[Tuesday, 6AM–6PM], [Wednesday, 6AM–6PM], [Th...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80dcdc23021a683d:0x51385c359da6b12e, 0x80dc...",https://www.google.com/maps/place//data=!4m2!3...
4,Alton Self Storage,"Alton Self Storage, 2215 Alton Pkwy, Irvine, C...",0x80dcde9ed485d8bb:0xadfae03de02d931d,None,33.692216,-117.837010,[Self-storage facility],4.3,118,None,"[[Tuesday, 9AM–6PM], [Wednesday, 9AM–6PM], [Th...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 9AM,"[0x80dcde8138ba81e5:0xb6d8b753a1b76050, 0x80dc...",https://www.google.com/maps/place//data=!4m2!3...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025006,Maven Realty,"Maven Realty, 2180 McMillan Ave #71672, North ...",0x88fe64e7daceaaab:0x101c046dcadcc9c3,None,32.857737,-79.985839,[Real estate agency],4.9,15,None,"[[Saturday, Open 24 hours], [Sunday, Open 24 h...",None,Open 24 hours,"[0x88fe657160ccc965:0xc675c849dc6d96ba, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...
3025007,Stone Castle Fence,"Stone Castle Fence, 124 Cypress Gardens Rd, Mo...",0x88fe59f4602c95fd:0x57e2b1904d9e6949,None,33.089951,-80.078021,"[Fence contractor, Construction company, Contr...",4.0,18,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...",None,Closed ⋅ Opens 7AM Mon,"[0x88fe8d1444299bbb:0x91daa2e82c59ea94, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...
3025008,Health Insurance Solutions Team,"Health Insurance Solutions Team, 6650 Rivers A...",0x88fe71f9617c15e1:0x89518c1d251706bf,None,32.918858,-80.027952,[Health insurance agency],5.0,18,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...","{'Service options': ['Online appointments'], '...",Closed ⋅ Opens 8AM Mon,"[0x88fe6fe45b8d3f15:0x41d4b940a40b246a, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...
3025009,Bernard Building Center,"Bernard Building Center, 27300 M-32, M-32, Hil...",0x4d34c7517d6e6e29:0x7fe553ee2f090cf0,None,45.056691,-83.894423,"[Building materials store, Electrical supply s...",4.7,25,None,"[[Saturday, 8AM–3PM], [Sunday, Closed], [Monda...","{'Service options': ['Curbside pickup', 'Deliv...",Closed ⋅ Opens 7:30AM Mon,"[0x4d335f6408d395d9:0xc20dea80d4c100d8, 0x4d34...",https://www.google.com/maps/place//data=!4m2!3...


Crear una nueva columna con las abreviaturas de los estados en metadata_sitios

In [5]:
metadata_sitios['states'] = metadata_sitios['address'].str.rsplit().str[-2]

# Rellenar los valores NaN/None con una cadena vacía para evitar errores
metadata_sitios['states'] = metadata_sitios['states'].fillna('')

# Filtrar las filas donde la longitud de la columna 'state' no es igual a 2 caracteres
condition = ~metadata_sitios['states'].str.match(r'^[A-Za-z]{2}$')
metadata_sitios = metadata_sitios[~condition]
metadata_sitios["states"].unique()


array(['GA', 'CA', 'CO', 'FL', 'UT', 'KS', 'MO', 'IL', 'TN', 'NC', 'IA',
       'VA', 'AZ', 'WA', 'HI', 'TX', 'KY', 'PA', 'LA', 'SC', 'NJ', 'NY',
       'WV', 'ND', 'OH', 'OR', 'IN', 'MI', 'MN', 'CT', 'MT', 'MA', 'NH',
       'DC', 'ID', 'SD', 'OK', 'MD', 'WY', 'NV', 'AR', 'RI', 'DE', 'NM',
       'AL', 'ME', 'WI', 'MS', 'NE', 'AK', 'VT', 'OZ', 'of', 'JJ', 'Ha',
       'at', 'Oh', 'OC', 'Fl', 'HP', 'Of', 'up', 'Me', 'Mi', 'SE', 'BB',
       'by', 'Sr', 'Dr', 'AB'], dtype=object)

In [6]:
metadata_sitios

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url,states
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...,GA
1,MemorialCare Breast Center - Irvine Sand Canyon,MemorialCare Breast Center - Irvine Sand Canyo...,0x80dcdd3b95c7284b:0x23d9b4440c982d37,None,33.668389,-117.765649,[Mammography service],4.7,33,None,"[[Tuesday, 8:30AM–5PM], [Wednesday, 8:30AM–5PM...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 8:30AM,"[0x80dcdc4aad8cb91f:0x62bdeb79a44d7633, 0x80dc...",https://www.google.com/maps/place//data=!4m2!3...,CA
2,U-Haul Neighborhood Dealer,"U-Haul Neighborhood Dealer, 1690 E 112th Ave, ...",0x876c769e600eda1d:0xdf1fffc2e00e1fe1,None,39.899189,-104.966765,"[Truck rental agency, Trailer rental service, ...",3.9,18,$,None,None,None,"[0x876c75d05d0e1bf3:0x1d2d9897d0dec9cb, 0x876c...",https://www.google.com/maps/place//data=!4m2!3...,CO
3,Nick's Auto Service,"Nick's Auto Service, 16291 Construction Cir E ...",0x80dcdc81a6d04bd3:0x72d18ce084849c10,None,33.694544,-117.821692,"[Auto repair shop, Brake shop, Car inspection ...",4.3,78,None,"[[Tuesday, 6AM–6PM], [Wednesday, 6AM–6PM], [Th...",{'Accessibility': ['Wheelchair accessible entr...,Open ⋅ Closes 6PM,"[0x80dcdc23021a683d:0x51385c359da6b12e, 0x80dc...",https://www.google.com/maps/place//data=!4m2!3...,CA
4,Alton Self Storage,"Alton Self Storage, 2215 Alton Pkwy, Irvine, C...",0x80dcde9ed485d8bb:0xadfae03de02d931d,None,33.692216,-117.837010,[Self-storage facility],4.3,118,None,"[[Tuesday, 9AM–6PM], [Wednesday, 9AM–6PM], [Th...",{'Accessibility': ['Wheelchair accessible entr...,Closed ⋅ Opens 9AM,"[0x80dcde8138ba81e5:0xb6d8b753a1b76050, 0x80dc...",https://www.google.com/maps/place//data=!4m2!3...,CA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3025006,Maven Realty,"Maven Realty, 2180 McMillan Ave #71672, North ...",0x88fe64e7daceaaab:0x101c046dcadcc9c3,None,32.857737,-79.985839,[Real estate agency],4.9,15,None,"[[Saturday, Open 24 hours], [Sunday, Open 24 h...",None,Open 24 hours,"[0x88fe657160ccc965:0xc675c849dc6d96ba, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...,SC
3025007,Stone Castle Fence,"Stone Castle Fence, 124 Cypress Gardens Rd, Mo...",0x88fe59f4602c95fd:0x57e2b1904d9e6949,None,33.089951,-80.078021,"[Fence contractor, Construction company, Contr...",4.0,18,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...",None,Closed ⋅ Opens 7AM Mon,"[0x88fe8d1444299bbb:0x91daa2e82c59ea94, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...,SC
3025008,Health Insurance Solutions Team,"Health Insurance Solutions Team, 6650 Rivers A...",0x88fe71f9617c15e1:0x89518c1d251706bf,None,32.918858,-80.027952,[Health insurance agency],5.0,18,None,"[[Saturday, Closed], [Sunday, Closed], [Monday...","{'Service options': ['Online appointments'], '...",Closed ⋅ Opens 8AM Mon,"[0x88fe6fe45b8d3f15:0x41d4b940a40b246a, 0x88fe...",https://www.google.com/maps/place//data=!4m2!3...,SC
3025009,Bernard Building Center,"Bernard Building Center, 27300 M-32, M-32, Hil...",0x4d34c7517d6e6e29:0x7fe553ee2f090cf0,None,45.056691,-83.894423,"[Building materials store, Electrical supply s...",4.7,25,None,"[[Saturday, 8AM–3PM], [Sunday, Closed], [Monda...","{'Service options': ['Curbside pickup', 'Deliv...",Closed ⋅ Opens 7:30AM Mon,"[0x4d335f6408d395d9:0xc20dea80d4c100d8, 0x4d34...",https://www.google.com/maps/place//data=!4m2!3...,MI


Filtrado por los estados elejidos 'PA' 'IL' 'DE' 'AZ' 'MO' 'NJ' 'FL' 'NV' 'CA' 'IN' 'TN' 'ID' 'LA' 'LV' 'MS' 'TX' 'KY'

In [7]:
# Lista de estados deseados
estados_elegidos = ['PA', 'IL', 'DE', 'AZ', 'MO', 'NJ', 'FL', 'NV', 'CA', 'IN', 'TN', 'ID', 'LA', 'LV', 'MS', 'TX', 'KY']

# Filtrar el DataFrame
metadata_sitios = metadata_sitios[metadata_sitios['states'].isin(estados_elegidos)]


Columnas a eliminar address, description, price, hours, MISC, state, relative_results

In [8]:
columns_to_drop = ['address', 'description', 'price', 'hours', 'MISC', 'state', 'relative_results']

metadata_sitios.drop(columns=columns_to_drop, inplace=True)

Filtrado por categoria 

In [9]:
metadata_sitios = metadata_sitios[metadata_sitios['category'].apply(lambda x: any('steak house' in word.lower() or 'steak' in word.lower() or 'bistec' in word.lower() or 'beef' in word.lower() for word in x) if isinstance(x, list) else False)]

In [10]:
metadata_sitios

,name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,url,states
11,Stagecoach Inn Restaurant and Lodge,0x87de5daa7cc7aefd:0x4220d709052263db,39.441171,-90.870254,"[Restaurant, Dessert shop, Hamburger restauran...",4.6,38,https://www.google.com/maps/place//data=!4m2!3...,IL
829,Charleys Philly Steaks,0x89c2fad0e50bf7cb:0x54444ad44d42f751,40.956903,-74.069185,"[Cheesesteak restaurant, American restaurant, ...",4.0,18,https://www.google.com/maps/place//data=!4m2!3...,NJ
840,Charleys Philly Steaks,0x89c3953fd908dfdd:0xdd16fbfbd5c00287,40.585621,-74.618851,"[Cheesesteak restaurant, American restaurant, ...",3.7,36,https://www.google.com/maps/place//data=!4m2!3...,NJ
4644,Ruth's Chris Steak House,0x8629d208ffeacbaf:0x34a7c3f19e96773b,32.435502,-90.148161,"[Restaurant, American restaurant, Caterer, Fin...",4.2,37,https://www.google.com/maps/place//data=!4m2!3...,MS
4771,Wawa,0x89c6d1b9c859e8d3:0x31cd6538a59b1652,39.806306,-75.094958,"[Convenience store, Breakfast restaurant, Cafe...",4.2,78,https://www.google.com/maps/place//data=!4m2!3...,NJ
...,...,...,...,...,...,...,...,...,...
3013483,Amazon Grill,0x88db4006b3cd75e9:0x333ebd044f215255,26.571300,-81.870963,"[Brazilian restaurant, Barbecue restaurant, Bu...",3.0,28,https://www.google.com/maps/place//data=!4m2!3...,FL
3014505,Jimmy's Island Grille,0x880e4ebce70f737f:0x212f786b3b381e0c,41.811042,-87.950310,"[Hawaiian restaurant, Catering food and drink ...",3.8,8,https://www.google.com/maps/place//data=!4m2!3...,IL
3014788,The Pennywort,0x808fcea4c120460b:0x24494b951e5b5716,37.406680,-121.886331,"[Vietnamese restaurant, Lunch restaurant, Stea...",4.5,28,https://www.google.com/maps/place//data=!4m2!3...,CA
3020254,The Republic Smokehouse Saloon,0x8640bf410343113b:0x96903b1ae150a9b2,29.752977,-95.375914,"[Barbecue restaurant, Bar & grill, Breakfast r...",3.3,48,https://www.google.com/maps/place//data=!4m2!3...,TX


Eliminar duplicados

In [11]:
# Convertir las columnas que contienen listas a cadenas
for col in metadata_sitios.columns:
    if metadata_sitios[col].apply(type).eq(list).any():
        metadata_sitios[col] = metadata_sitios[col].astype(str)

# Eliminar filas duplicadas
metadata_sitios = metadata_sitios.drop_duplicates()

# Convertir las columnas que contienen cadenas de listas de nuevo a listas
for col in metadata_sitios.columns:
    if metadata_sitios[col].apply(type).eq(str).any():
        try:
            metadata_sitios[col] = metadata_sitios[col].apply(eval)
        except:
            pass

In [12]:
metadata_sitios

,name,gmap_id,latitude,longitude,category,avg_rating,num_of_reviews,url,states
11,Stagecoach Inn Restaurant and Lodge,0x87de5daa7cc7aefd:0x4220d709052263db,39.441171,-90.870254,"[Restaurant, Dessert shop, Hamburger restauran...",4.6,38,https://www.google.com/maps/place//data=!4m2!3...,IL
829,Charleys Philly Steaks,0x89c2fad0e50bf7cb:0x54444ad44d42f751,40.956903,-74.069185,"[Cheesesteak restaurant, American restaurant, ...",4.0,18,https://www.google.com/maps/place//data=!4m2!3...,NJ
840,Charleys Philly Steaks,0x89c3953fd908dfdd:0xdd16fbfbd5c00287,40.585621,-74.618851,"[Cheesesteak restaurant, American restaurant, ...",3.7,36,https://www.google.com/maps/place//data=!4m2!3...,NJ
4644,Ruth's Chris Steak House,0x8629d208ffeacbaf:0x34a7c3f19e96773b,32.435502,-90.148161,"[Restaurant, American restaurant, Caterer, Fin...",4.2,37,https://www.google.com/maps/place//data=!4m2!3...,MS
4771,Wawa,0x89c6d1b9c859e8d3:0x31cd6538a59b1652,39.806306,-75.094958,"[Convenience store, Breakfast restaurant, Cafe...",4.2,78,https://www.google.com/maps/place//data=!4m2!3...,NJ
...,...,...,...,...,...,...,...,...,...
3013483,Amazon Grill,0x88db4006b3cd75e9:0x333ebd044f215255,26.571300,-81.870963,"[Brazilian restaurant, Barbecue restaurant, Bu...",3.0,28,https://www.google.com/maps/place//data=!4m2!3...,FL
3014505,Jimmy's Island Grille,0x880e4ebce70f737f:0x212f786b3b381e0c,41.811042,-87.950310,"[Hawaiian restaurant, Catering food and drink ...",3.8,8,https://www.google.com/maps/place//data=!4m2!3...,IL
3014788,The Pennywort,0x808fcea4c120460b:0x24494b951e5b5716,37.406680,-121.886331,"[Vietnamese restaurant, Lunch restaurant, Stea...",4.5,28,https://www.google.com/maps/place//data=!4m2!3...,CA
3020254,The Republic Smokehouse Saloon,0x8640bf410343113b:0x96903b1ae150a9b2,29.752977,-95.375914,"[Barbecue restaurant, Bar & grill, Breakfast r...",3.3,48,https://www.google.com/maps/place//data=!4m2!3...,TX


## Realizamos el ETL en los reviews de los estados seleccionados
Pensilvania (PA), Illinois (IL), Delaware (DE), Arizona (AZ), Misuri (MO), New Jersey (NJ), Florida (FL), Nevada (NV), California (CA), Indiana (IN), Tennessee (TN), Idaho (ID), Luisiana (LA), Louisiana (LV), Mississippi (MS), Texas (TX), Kentucky (KY) 

In [13]:
# Unimos los dataset de los estados selaccionados en uno solo

estados = (review_Pennsylvania, review_Illinois, review_Delaware, review_Arizona, review_Missouri, review_New_Jersey, review_Florida, review_Nevada, review_California, review_Indiana, review_Tennessee, review_Idaho, review_Louisiana, review_Mississippi, review_Texas, review_Kentucky)

reviews_estados = pd.concat(estados)

In [14]:
reviews_estados

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.025199e+20,Ethan Fisher,1583900221131,5,The guys and I went here over spring break and...,None,None,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
1,1.049313e+20,Casey W,1554686595304,4,"Good crowd, great bar food for late night crav...",[{'url': ['https://lh5.googleusercontent.com/p...,None,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
2,1.132730e+20,April Ulmer,1595367961383,5,The food is pretty okay but the drinks are GRE...,None,None,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
3,1.026472e+20,Janelle Morehart,1599360401023,5,Great place! Amazing staff!,None,None,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
4,1.063430e+20,Jeffrey Kuenzi,1582548467722,5,Reasonable prices. Lots of bar games. They ha...,None,None,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
...,...,...,...,...,...,...,...,...
1649995,1.023330e+20,michael bennett,1539831340846,5,Very helpful organization LIVE THEM,None,None,0x8869a10089a88197:0x1730907d79ad2df3
1649996,1.135018e+20,Michelle Corbett,1618762835646,3,None,None,None,0x8869a10089a88197:0x1730907d79ad2df3
1649997,1.163143e+20,LISA LEWIS,1529720825049,5,None,None,None,0x8869a10089a88197:0x1730907d79ad2df3
1649998,1.034908e+20,Kathleen Perkins Second,1525566964705,1,None,None,None,0x8869a10089a88197:0x1730907d79ad2df3


In [15]:
# Columnas a eliminar name, resp

columnas = ['name','resp']

reviews_estados.drop(columns=columnas, inplace=True)

In [16]:
# Convertir las columnas que contienen listas a cadenas
for col in reviews_estados.columns:
    if reviews_estados[col].apply(type).eq(list).any():
        reviews_estados[col] = reviews_estados[col].astype(str)

# Eliminar filas duplicadas
reviews_estados = reviews_estados.drop_duplicates()

# Convertir las columnas que contienen cadenas de listas de nuevo a listas
for col in reviews_estados.columns:
    if reviews_estados[col].apply(type).eq(str).any():
        try:
            reviews_estados[col] = reviews_estados[col].apply(eval)
        except:
            pass

In [17]:
reviews_estados

,user_id,time,rating,text,pics,gmap_id
0,1.025199e+20,1583900221131,5,The guys and I went here over spring break and...,None,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
1,1.049313e+20,1554686595304,4,"Good crowd, great bar food for late night crav...",[{'url': ['https://lh5.googleusercontent.com/p...,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
2,1.132730e+20,1595367961383,5,The food is pretty okay but the drinks are GRE...,None,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
3,1.026472e+20,1599360401023,5,Great place! Amazing staff!,None,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
4,1.063430e+20,1582548467722,5,Reasonable prices. Lots of bar games. They ha...,None,0x89cfa6188ac26cbf:0x7e97f2b4b6f7955b
...,...,...,...,...,...,...
1649995,1.023330e+20,1539831340846,5,Very helpful organization LIVE THEM,None,0x8869a10089a88197:0x1730907d79ad2df3
1649996,1.135018e+20,1618762835646,3,None,None,0x8869a10089a88197:0x1730907d79ad2df3
1649997,1.163143e+20,1529720825049,5,None,None,0x8869a10089a88197:0x1730907d79ad2df3
1649998,1.034908e+20,1525566964705,1,None,None,0x8869a10089a88197:0x1730907d79ad2df3


In [18]:
# Crear una serie booleana que es True si el gmap_id de reviews_estados está también en metadata_sitios
is_in_metadata = reviews_estados['gmap_id'].isin(metadata_sitios['gmap_id'])

# Sumar los True para obtener el número de filas que cumplen la condición
count_common_gmap_ids = is_in_metadata.sum()

print(count_common_gmap_ids)


92105


In [19]:
reviews_steakhouse = pd.merge(reviews_estados, metadata_sitios[['gmap_id']], on='gmap_id', how='inner')
reviews_steakhouse

,user_id,time,rating,text,pics,gmap_id
0,1.171925e+20,1617212125778,5,Some of the stuff here is amazing! ( Chris is...,None,0x89c15298cba09275:0xfffb76888e28941b
1,1.153772e+20,1498054826727,5,Love the new look inside,[{'url': ['https://lh5.googleusercontent.com/p...,0x89c15298cba09275:0xfffb76888e28941b
2,1.106018e+20,1617599245816,1,"For the last three years, every half gallon of...",None,0x89c15298cba09275:0xfffb76888e28941b
3,1.128984e+20,1606156481360,5,What can I say? Some are large with gas pumps ...,None,0x89c15298cba09275:0xfffb76888e28941b
4,1.170222e+20,1535127908778,1,"It use to be a really good staff here, now not...",None,0x89c15298cba09275:0xfffb76888e28941b
...,...,...,...,...,...,...
92100,1.180476e+20,1607129550040,1,No good. They take so long for delivery the fo...,None,0x8841c7bf568a25df:0xee97c287496866f8
92101,1.029845e+20,1617489961490,5,None,None,0x8841c7bf568a25df:0xee97c287496866f8
92102,1.148514e+20,1617782484760,4,None,None,0x8841c7bf568a25df:0xee97c287496866f8
92103,1.164074e+20,1618660394388,5,None,None,0x8841c7bf568a25df:0xee97c287496866f8


In [20]:
# Guardar reviews_estados en formato Parquet
reviews_estados.to_parquet('reviews_estados.parquet')

# Guardar metadata_sitios en formato Parquet
metadata_sitios.to_parquet('metadata_sitios.parquet')

# Guardar reviews_steakhouse en formato Parquet
reviews_steakhouse.to_parquet('reviews_steakhouse.parquet')
